In [9]:
import pandas as pd


In [ ]:
# VORP positional values for bench and waiver

vorp_bench_idx = {
    'RB': 22,     
    'WR': 18,
    'TE': 8,
    'QB': 8,
    'DST': 8,
    'K': 8
}

vorp_waiver_idx = {
    'RB': 42,   # 40-44
    'WR': 46,   # 44-48
    'TE': 11,
    'QB': 12,
    'DST': 8,
    'K': 8
}


In [39]:
df = pd.read_csv('data/projections_2025_wk0.csv', index_col=0)
df.head(10)

,player,position,team,points,sd_pts,dropoff,floor,ceiling,points_vor,floor_vor,ceiling_vor,rank,floor_rank,ceiling_rank,position_rank,tier,adp,aav,uncertainty
1,Saquon Barkley,RB,PHI,278.0,11.5,7.520,264.0,295.0,164.0,181.0,155.0,1,1.0,2.0,1,1,5.30,55.4,1.0
2,Bijan Robinson,RB,ATL,271.0,17.3,10.100,251.0,301.0,157.0,168.0,160.0,2,2.0,1.0,2,1,3.54,59.4,10.0
3,Jahmyr Gibbs,RB,DET,261.0,13.7,9.320,244.0,283.0,147.0,160.0,142.0,3,3.0,3.0,3,2,6.36,51.5,1.0
4,Derrick Henry,RB,BAL,251.0,20.6,21.300,217.0,283.0,137.0,134.0,142.0,4,4.0,4.0,4,3,11.30,47.8,15.0
5,Josh Jacobs,RB,GB,230.0,20.2,3.580,197.0,255.0,116.0,114.0,115.0,5,7.0,6.0,5,3,19.00,38.6,24.0
6,Christian McCaffrey,RB,SF,227.0,17.3,1.210,203.0,246.0,112.0,120.0,105.0,6,5.0,9.0,6,3,10.90,55.7,13.0
7,Jonathan Taylor,RB,IND,225.0,27.2,2.470,194.0,265.0,111.0,111.0,124.0,7,8.0,5.0,7,3,17.60,44.8,61.0
8,Ja'Marr Chase,WR,CIN,238.0,18.9,25.500,215.0,265.0,110.0,107.0,109.0,8,9.0,7.0,1,1,2.93,59.6,21.0
9,Devon Achane,RB,MIA,223.0,12.6,3.250,199.0,235.0,109.0,116.0,94.1,9,6.0,13.0,8,4,16.60,37.7,3.0
10,Ashton Jeanty,RB,LVR,220.0,18.4,0.892,185.0,242.0,105.0,102.0,102.0,10,10.0,10.0,9,4,12.10,71.6,14.0


In [ ]:
def nth_by_position(df: pd.DataFrame, idx_map: dict, sort_col: str = "points") -> pd.DataFrame:
    """
    Return the nth (per position) highest rows based on sort_col.
    idx_map: e.g. {'RB': 22, 'WR': 18, 'TE': 8, 'QB': 8, 'D/ST': 8, 'K': 8}
    Sorts descending by sort_col (higher is better).
    """
    # ensure numeric sorting if needed
    df = df.copy()
    df[sort_col] = pd.to_numeric(df[sort_col], errors="coerce")

    # sort once
    df_sorted = df.sort_values(["position", sort_col], ascending=[True, False])

    rows = []
    for pos, grp in df_sorted.groupby("position", sort=False):
        n = idx_map.get(pos)
        if n is None:
            continue  # skip positions not in the map
        n_clamped = max(1, min(int(n + 1), len(grp)))   # clamp to available players
        rows.append(grp.iloc[n_clamped - 1])

    return pd.DataFrame(rows).reset_index(drop=True)



In [43]:
bench_baselines = nth_by_position(df, vorp_bench_idx, sort_col="points")
waiver_baselines = nth_by_position(df, vorp_waiver_idx, sort_col="points")

# If you just want a quick lookup of baseline points per position:
bench_points = bench_baselines.set_index("position")["points"].to_dict()
waiver_points = waiver_baselines.set_index("position")["points"].to_dict()

# Example VORP column using bench baseline:
df["points_vor_bench"] = df.apply(
    lambda r: r["points"] - bench_points.get(r["position"], float("nan")), axis=1
)

bench_points

{'K': 147.0, 'QB': 289.0, 'RB': 271.0, 'TE': 96.5, 'WR': 154.0}